In [ ]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import matplotlib.pyplot as plt
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))

ipy = get_ipython()
#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
from os.path import join as pjoin
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)


#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
#scripts_dir = pjoin(data_dir_general,'full_experiments','scripts2')

data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'previous_analyses')


print(data_dir_input, scripts_dir)

subjects = [f for f in os.listdir(data_dir_input) if f.startswith('sub') ]
subjects = list(sorted(subjects))
print(subjects)

from config2 import *

from base2 import (width, height, radius, calc_target_coordinates_centered,
                  calc_rad_angle_from_coordinates)

import numpy as np
target_angs = (np.array([157.5, 112.5, 67.5, 22.5]) + 90) * \
              (np.pi/180)
target_coords = calc_target_coordinates_centered(target_angs)


In [ ]:
dfs = []
for subj in subjects:
    task = 'VisuoMotor'
    fname = pjoin(path_data, subj, 'behavdata',
                    f'behav_{task}_df.pkl' )
    behav_df = pd.read_pickle(fname)
    behav_df['subject'] = subj
    dfs += [behav_df]
behav_df = pd.concat(dfs)

In [ ]:
behav_df.reset_index(inplace=True)
behav_df.drop('index',1,inplace=True)

In [ ]:
#set( behav_df['trials'] )

In [ ]:
from error_sensitivity import computeErrSens2

dfc = behav_df[behav_df['subject'] == subj]
trial_inds = np.arange(len(dfc) )

non_hit_not_adj, df_esv, vndef2vn  =\
    computeErrSens2(dfc, trial_inds, enforce_consistency = 0)

In [ ]:
df_esv_all = []
non_hit_not_adj_per_subj = {}
for subj in subjects:
    dfc = behav_df[behav_df['subject'] == subj]
    trial_inds = np.arange(len(dfc) )

    non_hit_not_adj, df_esv, vndef2vn  =\
        computeErrSens2(dfc, trial_inds, enforce_consistency = 0)    
    
    df_esv['subject'] = subj
    #adjustNonHit(non_hit_not_adj, env, time_locked) 
    
    non_hit_not_adj_per_subj[subj] = non_hit_not_adj
    df_esv_all += [df_esv]
df_esv_all = pd.concat(df_esv_all)
df_esv_all.reset_index(inplace=True)
df_esv_all.drop('index',1,inplace=True)

In [ ]:
dfc

In [ ]:
df_esv

In [ ]:
dfc

In [ ]:
set( df_esv_all['perturbation'] )

In [ ]:
# ax = plt.gca()
# ax.scatter(df_esv['movement'], )

In [ ]:
df_esv_all_p = df_esv_all.drop('prev_target',1)
grp = df_esv_all_p.groupby('target')

In [ ]:
for tgt in grp.groups.keys():
    grp.get_group(tgt)

In [ ]:
grp.get_group(list(grp.groups.keys())[0])

In [ ]:
import seaborn as sns

In [ ]:
env

In [ ]:
env2envcode

In [ ]:
ab

In [ ]:
a,b = list( zip( *list( env2envcode.items() ) ) )
envcode2env = dict( zip( b,a ) )

In [ ]:
a,b = list( zip( *env2envcode.items() ) )
dict( *zip(b,a) )

In [ ]:
env2envcode[envc]

In [ ]:
df_esv_all[df_esv_all['perturbation'] == 0].drop('perturbation',1)

In [ ]:
df_esv_all[ (df_esvc['environment'] == envc) & df_esv_all['non_hit'] ]

In [ ]:
np.max( df_esv_all.loc[df_esv_all['non_hit'], 'err_sens'] )

In [ ]:
type( df_esv_all['perturbation'].iloc[1] )

### merge tgts

In [ ]:

df_esvc = df_esv_all.drop('prev_target',axis=1)
df_esvc = df_esvc.drop('target',axis=1)
for envc in [0,1]:
    df_esvc_curenv = df_esvc[(df_esvc['environment'] == envc) & df_esvc['non_hit']]
    df_esvc_curenv = df_esvc_curenv.drop('environment',axis=1)        
    df_esvc_curenv = df_esvc_curenv.drop('non_hit',axis=1)        
    df_esvc_curenv = df_esvc_curenv.drop('subject',axis=1)  
    df_esvc_curenv = df_esvc_curenv[df_esvc_curenv['err_sens'].apply(np.abs) < 10]

    sns.pairplot(df_esvc_curenv, hue='perturbation')
    plt.suptitle(f'tgt = {tgt:.3f} env={envcode2env[envc]}' )
#         break
#     break
#plt.close()

### separate tgts

In [ ]:
for tgt in list( set( df_esv_all['target'] ) )[:2]:
    df_esvc = df_esv_all[df_esv_all['target'] == tgt].drop('prev_target',axis=1)
    df_esvc = df_esvc.drop('target',axis=1)
    for envc in [0,1]:
        df_esvc_curenv = df_esvc[(df_esvc['environment'] == envc) & df_esvc['non_hit']]
        df_esvc_curenv = df_esvc_curenv.drop('environment',axis=1)        
        df_esvc_curenv = df_esvc_curenv.drop('non_hit',axis=1)        
        df_esvc_curenv = df_esvc_curenv.drop('subject',axis=1)  
        df_esvc_curenv = df_esvc_curenv[df_esvc_curenv['err_sens'].apply(np.abs) < 10]
        
        sns.pairplot(df_esvc_curenv, hue='perturbation')
        plt.suptitle(f'tgt = {tgt:.3f} env={envcode2env[envc]}' )
#         break
#     break
    #plt.close()

In [ ]:
for tgt in list( set( df_esv_all['target'] ) )[:2]:
    df_esvc = df_esv_all[df_esv_all['target'] == tgt].drop('prev_target',axis=1)
    df_esvc = df_esvc.drop('target',1)
    for envc in [0,1]:
        df_esvc_curenv = df_esvc[(df_esvc['environment'] == envc) & df_esvc['non_hit']]
        df_esvc_curenv = df_esvc_curenv.drop('environment',axis=1)        
        df_esvc_curenv = df_esvc_curenv.drop('non_hit',axis=1)        
        df_esvc_curenv = df_esvc_curenv.drop('subject',axis=1)  
        df_esvc_curenv = df_esvc_curenv[df_esvc_curenv['err_sens'].apply(np.abs) < 10]
        
        sns.pairplot(df_esvc_curenv, kind='hist')
        plt.suptitle(f'tgt = {tgt:.3f} env={envcode2env[envc]}' )
#         break
#     break
    #plt.close()

In [ ]:
sns.pairplot(df_esv_all)

# Err per subject

In [ ]:
trial_inds = np.arange(len(behav_df) )

from error_sensitivity import computeErrSens2
non_hit_not_adj, df_esv, vndef2vn  =\
    computeErrSens2(behav_df, trial_inds, enforce_consistency = 0)


df_esv

In [ ]:
#si = 2
#subj = subjects[si]
valss = []
for subj in subjects:
    task = 'VisuoMotor'
    fname = pjoin(path_data, subj, 'behavdata',
                    f'behav_{task}_df.pkl' )
    behav_df = pd.read_pickle(fname)


    trial_inds = np.arange(len(behav_df) )

    from error_sensitivity import computeErrSens2
    non_hit_not_adj, df_esv, vndef2vn  =\
        computeErrSens2(behav_df, trial_inds, enforce_consistency = 0)


    vals = behav_df['error']
    valss += [vals]
    plt.plot(vals)
    plt.plot( behav_df['perturbation'] / 50 )
    plt.show()
    plt.close()
    

In [ ]:
valss = np.array(valss)
plt.plot( valss.mean(0) )
plt.plot( behav_df['perturbation'] / 160)

# correction

In [ ]:
#si = 2
#subj = subjects[si]
valss = []
for subj in subjects:
    task = 'VisuoMotor'
    fname = pjoin(path_data, subj, 'behavdata',
                    f'behav_{task}_df.pkl' )
    behav_df = pd.read_pickle(fname)


    trial_inds = np.arange(len(behav_df) )

    from error_sensitivity import computeErrSens2
    non_hit_not_adj, df_esv, vndef2vn  =\
        computeErrSens2(behav_df, trial_inds, enforce_consistency = 0)


    vals = df_esv['correction'][3:]
    valss += [vals]
    plt.plot(vals)
    plt.plot( behav_df['perturbation'] / 50 )
    plt.show()
    plt.close()
    

In [ ]:
valss = np.array(valss)
plt.plot( valss.mean(0) )
plt.plot( behav_df['perturbation'] / 160)

# err sens per subject

In [ ]:
#si = 2
#subj = subjects[si]
err_senss = []
for subj in subjects:
    task = 'VisuoMotor'
    fname = pjoin(path_data, subj, 'behavdata',
                    f'behav_{task}_df.pkl' )
    behav_df = pd.read_pickle(fname)


    trial_inds = np.arange(len(behav_df) )

    from error_sensitivity import computeErrSens2
    non_hit_not_adj, df_esv, vndef2vn  =\
        computeErrSens2(behav_df, trial_inds, enforce_consistency = 0)


    err_sens = df_esv['err_sens']
    err_senss += [err_sens]
    plt.plot(err_sens)
    plt.plot( behav_df['perturbation'] )
    plt.show()
    plt.close()
    

In [ ]:
err_senss = np.array(err_senss ) 

In [ ]:
err_senss.shape

In [ ]:
set(behav_df['perturbation'])

In [ ]:
pvals = [-30,0,30]
#pvals = [0]
for pv in pvals:
    safe = 0.1
    c = (behav_df['perturbation'] >= pv-safe ) & (behav_df['perturbation'] <= pv+safe)
    c &=  ~df_esv['err_sens'].isin([np.inf, -np.inf, np.nan])
    m = df_esv[c]['err_sens'].mean()
    std = df_esv[c]['err_sens'].std()
    print(pv,m,std)

In [ ]:
from error_sensitivity import computeErrSens2
non_hit_not_adj, df_esv, vndef2vn  =\
    computeErrSens2(behav_df, trial_inds, enforce_consistency = 0)


df_esv

In [ ]:
df_esv[c]['err_sens'].isin([-np.inf])

In [ ]:
plt.plot( err_senss.mean(0) )
plt.plot( behav_df['perturbation'] )

In [ ]:
si = 2
subj = subjects[si]
behav_data_dir = pjoin(data_dir_input,'behavdata')
#behavdata

task = 'VisuoMotor_'
fname = pjoin(data_dir_input, subj, 'behavdata',
                'behav_%sdf.pkl' % task)


behav_df = pd.read_pickle(fname)



targets      = np.array(behav_df['target'])
environment  = np.array(behav_df['environment'])
feedback     = np.array(behav_df['feedback'])
feedbackX    = np.array(behav_df['feedbackX'])
feedbackY    = np.array(behav_df['feedbackY'])
org_feedback = np.array(behav_df['org_feedback'])
trajectoryX = np.array(behav_df['trajectoryX'])
trajectoryY = np.array(behav_df['trajectoryY'])
errors       = np.array(behav_df['error'])
belief       = np.array(behav_df['belief'])
perturbations       = np.array(behav_df['perturbation'])
err       = np.array(behav_df['err'])

f = calc_target_coordinates_centered(org_feedback)
org_feedbackXY = np.array(f)
org_feedbackX = org_feedbackXY[:,0]
org_feedbackY = org_feedbackXY[:,1]

endptsX = np.array( [traj[-1] for traj in trajectoryX] )
endptsY = np.array( [traj[-1] for traj in trajectoryY] )
endpts = calc_rad_angle_from_coordinates(endptsX,endptsY)

endpts_2D = np.array([endptsX,endptsY] )
feedback_2D = np.array([feedbackX,feedbackY] )
org_feedback_2D = np.array([org_feedbackX,org_feedbackY] )

#############  load err sens

err_sens_per_env = {}
for env in ['stable','random']:
    freq_name = 'broad'
    hpass = '0.1'  # '0.1', no_hpass
    output_folder = 'corr_spoc_es2_%s' % hpass
    results_folder = pjoin(data_dir_input,subj,'results', output_folder)
    fname = pjoin(results_folder,
                    '%s_%ses_%s.npy' % (subj, env, freq_name))
    print(f'Saving to {fname}')
    
    err_sens_per_env[env] = np.load(fname)


## plot err sens

In [ ]:
for env in ['stable','random']:
    plt.figure()
    err_sens = err_sens_per_env[env]
    plt.plot( err_sens )
    me = np.mean(err_sens)
    std = np.std(err_sens)
    plt.axhline( me, ls=':',c='r' )
    plt.axhline( me-std, ls=':',c='g' )
    plt.axhline( me+std, ls=':',c='g' )
    plt.title(env)
    plt.ylim(-5,5)
    #plt.close()
    

In [ ]:
plt.plot(  feedback - target_angs[targets] )
#plt.plot( -  feedback + target_angs[targets] )
err_mod = np.minimum(err, 180-err)
plt.plot( err_mod / 180 * np.pi)

In [ ]:
np.diff( org_feedback - target_angs[targets] )

In [ ]:
feedback - target_angs[targets]

In [ ]:
fname_behavior

In [ ]:
#targets

In [ ]:
df_esv

In [ ]:
environment_change = np.diff(environment)
np.where(environment_change > 0)[0]

In [ ]:
nr = 2; nc = 2
ww = 5; hh = 3
col_timeplot = 1
col_spaceplot=  0
fig,axs = plt.subplots(nr,nc,figsize=(nc*ww,nr*hh))
ax = axs[0,col_timeplot]
ax.plot(perturbations)
ax.set_title('pert')

ax = axs[1,col_timeplot]
ax.plot(errors)
ax.set_title('err')

for targ in target_coords:
    axs[0,col_spaceplot].plot(targ[0], targ[1], 'ro', markersize=15)
for i in range(len(trajectoryX)):
    axs[0,col_spaceplot].plot(trajectoryX[i], trajectoryY[i], c='blue', alpha=0.3)
    
    
# for i in range(len(trajectoryX)):
#     rotx, roty = 0,0
#     axs[0,col_spaceplot].plot(trajectoryX[i], trajectoryY[i], c='red', alpha=0.3)


In [ ]:
plt.figure(figsize=(12,4))
plt.plot( feedback - org_feedback )
plt.plot(feedback)
plt.plot(perturbations / 18)
plt.plot(3 + org_feedback)
plt.plot(6 +endpts)

In [ ]:
plt.plot(endpts - feedback, label='endpts - feedback')
plt.plot(endpts - org_feedback, label='endpts - org_feedback')
plt.legend()

In [ ]:
np.mean(np.abs(endpts - org_feedback) ),np.std(np.abs(endpts - org_feedback) )

In [ ]:
#degrees
np.mean(np.abs(endpts - org_feedback) )/np.pi * 180,np.std(np.abs(endpts - org_feedback) ) * 180/np.pi

In [ ]:
apert = np.abs(perturbations)
plt.plot(apert)
plt.plot(apert > 1e-10)
trial_inds = np.where(apert)[0]
plt.plot(trial_inds,np.ones(len(trial_inds)),lw=0,marker='.')

In [ ]:
# check random part
assert len(perturbations) == len(feedback)
plt.figure(figsize=(13,3))
#apert = np.abs(perturbations)
trial_inds = np.where( environment)[0]
d = (feedback - org_feedback)[ trial_inds]
plt.plot(trial_inds,d,lw=0,marker='.')
#plt.plot(trial_inds,perturbations[trial_inds],label='pert',lw=0,marker='.')
plt.legend()

In [ ]:
plt.figure(figsize=(13,3))
random_errors = perturbations*environment
plt.plot(random_errors)
plt.plot(perturbations)

In [ ]:
plt.hist(d)

In [ ]:
# see that target indices change
plt.plot(range(len(targets)), targets, lw=0, marker='o')

In [ ]:
pert_change_inds = np.where( np.abs( np.diff(perturbations) ) > 1e-10 )[0]
#perturbation[pert_change_inds[0]], perturbation[pert_change_inds[0]+1]
pert_change_inds += 1
print(pert_change_inds)
perturbations[pert_change_inds[0]-1], perturbations[pert_change_inds[0]]

In [ ]:
plt.figure(figsize=(12,3))
ax=plt.gca()
factor = 1
ax.plot(errors * factor, label=f'err * {factor}')
ax.plot(perturbations / 30, label='pert')
ax.set_title('err and pert')
ax.set_ylim(-1.3,1.3)
ax.legend()

In [ ]:
plt.figure(figsize=(12,3))
ax=plt.gca()
factor = 1
ax.plot(errors * factor, label=f'err')
ax.plot(random_errors, label='pert', alpha=0.6)
ax.set_title('err and random err')
#ax.set_ylim(-50,60)
ax.legend()

In [ ]:
angles = np.degrees(np.arctan2(
                   (feedbackY - int(round(height/2.0)))/float(radius),
                   (feedbackX - int(round(width/2.0)))/float(radius))) + \
                   360


## Per target (pert lumped)

In [ ]:

from numpy import cos,sin
plt.figure(figsize=(7,12))
tgt_shifts = np.arange(len(target_angs) ) * 320
for trial,tgtind in enumerate(targets):
    Xs = trajectoryX[trial]
    Ys = trajectoryX[trial]
    cta = target_angs[tgtind]
    trajX_rot = Xs * cos(-cta / 360 * 2 * np.pi) - Ys * sin(cta / 360 * 2 * np.pi)
    trajY_rot = Xs * sin(-cta / 360 * 2 * np.pi) + Ys * cos(cta / 360 * 2 * np.pi)
    trajY_rot += tgt_shifts[tgtind]

    plt.plot(trajX_rot,trajY_rot)

## traj endpots vs feedback

In [ ]:
# traj endpots are related to org feedback but still have errors
import scipy
d1 = scipy.linalg.norm(endpts_2D - feedback_2D,     axis=0)
d2 = scipy.linalg.norm(endpts_2D - org_feedback_2D, axis=0)

plt.plot(d1, label='feedback')
plt.plot(d2,      alpha=0.6, c='y', label='org_feedback')
plt.legend()

## endpoints

In [ ]:
cs = (np.arange(len(targets ))+1) / len(targets)
cs = 'blue'
plt.scatter(endptsX,endptsY, c= cs, cmap='viridis')

# plot trajectories per target

In [ ]:
def plotTrialHor(ax,trial):    
    Xs = trajectoryX[trial]
    Ys = trajectoryY[trial]
    cta = target_angs[tgtind]
    cta = -cta
    X0 = Xs[0]
    Y0 = Ys[0]
    trajX_rot = (Xs - X0) * cos(cta ) - (Ys-Y0) * sin(cta ) + X0
    trajY_rot = (Xs - X0) * sin(cta ) + (Ys-Y0) * cos(cta ) + Y0
    trajY_rot += tgt_shifts[tgtind]
    
    ax.plot(trajX_rot,trajY_rot, c='blue', alpha=0.3)

    tx = radius #targetAngs[tgtind]
    ty = tgt_shifts[tgtind]
    #tx_rot = tx * cos(-cta / 360 * 2 * np.pi) - ty * sin(cta / 360 * 2 * np.pi)
    #ty_rot = tx * sin(-cta / 360 * 2 * np.pi) + ty * cos(cta / 360 * 2 * np.pi)
    tx_rot = tx
    ty_rot = ty
    ax.plot([tx_rot], [ty_rot], 'ro', markersize=15)

    ax.axhline(tgt_shifts[tgtind], ls=':', c='black')
    
def plotTrial(ax,trial):    
    Xs = trajectoryX[trial]
    Ys = trajectoryY[trial]    
    tx,ty = target_coords[targets[trial]]
    ax.plot([tx], [ty], 'ro', markersize=15)
    fbx,fby = feedbackX[trial],feedbackY[trial]
    ax.plot([fbx], [fby], c='yellow', marker='o', markersize=10, alpha=0.5)
    
    fbxo,fbyo = org_feedbackX[trial],org_feedbackY[trial]
    ax.plot([fbxo], [fbyo], c='green', marker='o', markersize=2)
    
    ax.scatter([0,tx], [0,ty], ls=':', c='black')    
    ax.plot(Xs,Ys, c='blue', alpha=0.3)
    
#     cta = target_angs[tgtind]
#     cta = -cta
#     X0 = Xs[0]
#     Y0 = Ys[0]
#     trajX_rot = (Xs - X0) * cos(cta ) - (Ys-Y0) * sin(cta ) + X0
#     trajY_rot = (Xs - X0) * sin(cta ) + (Ys-Y0) * cos(cta ) + Y0
#     trajY_rot += tgt_shifts[tgtind]
    
#     ax.plot(trajX_rot,trajY_rot, c='blue', alpha=0.3)

#     tx = radius #targetAngs[tgtind]
#     ty = tgt_shifts[tgtind]
#     #tx_rot = tx * cos(-cta / 360 * 2 * np.pi) - ty * sin(cta / 360 * 2 * np.pi)
#     #ty_rot = tx * sin(-cta / 360 * 2 * np.pi) + ty * cos(cta / 360 * 2 * np.pi)
#     tx_rot = tx
#     ty_rot = ty
#     ax.plot([tx_rot], [ty_rot], 'ro', markersize=15)


In [ ]:
perturbations_int = (perturbations.copy() ).astype(int)
#perturbations_int = (perturbations.copy() / np.pi * 180 ).astype(int)
pertu = np.unique(perturbations_int[environment == 0])
pert2ind = dict( zip( pertu, np.arange(len(pertu))) )
#pert2ind

nr = 4; nc = len(pertu) + 1
ww = 4; hh = 4
col_timeplot = 1
col_spaceplot=  0
fig,axs = plt.subplots(nr,nc,figsize=(nc*ww,nr*hh))
axs = axs.reshape((nr,nc))

from numpy import cos,sin
#plt.figure(figsize=(7,12))
tgt_shifts = np.arange(len(target_angs) ) * 320
enum_targets = np.array( list(enumerate(targets) ) )

for pert_val in pert2ind:
    inds_per_pertval = np.where(perturbations_int == pert_val)[0]
    for trial,tgtind in enum_targets[inds_per_pertval]:
        #pert_val = int( perturbations[trial] )
        colind = pert2ind[pert_val]
        ax = axs[tgtind,colind]
        ax.set_title(f'{pert_val}')
        plotTrial(ax,trial)

inds_per_pertval = np.where(environment)[0]
for trial,tgtind in enum_targets[inds_per_pertval]:
    colind = -1
    ax = axs[tgtind,colind]
    ax.set_title('random')
    plotTrial(ax,trial)
        
#fig.suptitle('Traj rotated by -<target angle>')
fig.suptitle('Traj')
plt.savefig(f'{subj}_reach_traj.pdf')